In [1]:
import pickle
import shutil
import pandas as pd
from tqdm import tqdm
from glob import glob

In [3]:
# Renaming _result.pkl files
detections =  glob('great_ape_detector_unlabelled/**/*.pkl', recursive=True)

for d in detections:
    name = d.split('/')[-1].split('_')[0] + '.pkl'
    shutil.move(d, f"great_ape_detector_unlabelled/{name}")

In [4]:
# Formatting detections
detections =  glob('great_ape_detector_unlabelled/**/*.pkl', recursive=True)

for det in tqdm(detections):
    store = []
    name = det.split('/')[-1].split('.')[0]

    with open(det, 'rb') as handle:
        d = pickle.load(handle)
    
    for i, elem in enumerate(d):
        x = {}
        x['filename']= f'{name}_frame_{i}.jpg'
        x['result'] = elem

        store.append(x)
    
    with open(det, 'wb') as handle:
        pickle.dump(store, handle, pickle.HIGHEST_PROTOCOL)

100%|██████████| 4187/4187 [00:10<00:00, 388.85it/s]


In [8]:
# Remove none core videos
with open('/home/dl18206/Desktop/phd/data/core_videos.pickle', 'rb') as handle:
    core_videos = pickle.load(handle)

core_videos = pd.DataFrame(core_videos, columns=['name'])
core_videos.name = core_videos.name.str.split('.').str[0]

detections = pd.DataFrame(glob('core_detections/**/*.pkl', recursive=True), columns=['name'])
detections.name = detections.name.str.split('/').str[-1].str.split('.').str[0]

trans = pd.read_csv('/home/dl18206/Desktop/phd/data/panaf/translation/core_video_translation.csv')
core_acp_videos = trans[trans.obfu_id.isin(core_videos.name)]['zooniverse_id'].values

for d in detections.name.values:
    name = d.split('/')[-1].split('.')[0]
    if(f"{name}" not in core_acp_videos):
        shutil.move(f"great_ape_detector_unlabelled/{d}.pkl", f"other_detections/{name}.pkl")

In [5]:
core_detections = [x.split('/')[-1].split('.')[0] for x in glob('core_detections/**/*.pkl', recursive=True)]
other_detections = list(set(glob('great_ape_detector_unlabelled/**/*.pkl', recursive=True)))

In [6]:
for d_i in tqdm(other_detections):
    if d_i.split('/')[-1].split('.')[0] not in core_detections:
        shutil.move(d_i, 'other_detections/')

100%|██████████| 4187/4187 [00:00<00:00, 8967.55it/s]
